# 実践演習 Day 1：streamlit と FastAPI のデモ

このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Face からモデルを用いた Streamlit のデモアプリ
- FastAPI と ngrok を使用した API の公開方法

演習を始める前に、HuggingFace と ngrok のアカウントを作成し、
それぞれの API トークンを取得する必要があります。

演習の時間では、以下の 3 つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2 つ目や 3 つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項

「02_streamlit_app」と「03_FastAPI」では、GPU を使用します。

これらを実行する際は、Google Colab 画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---


# 環境変数の設定（1~3 共有）


GitHub から演習用のコードを Clone します。


In [ ]:
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

必要な API トークンを.env に設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。

```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```

ダブルクオーテーションで囲まれた文字列を Huggingface のアクセストークンと、ngrok の認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下の URL からそれぞれのトークンを取得できます。

- Huggingface のアクセストークン
  https://huggingface.co/docs/hub/security-tokens

- ngrok の認証トークン
  https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルの PC にダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPU から GPU の利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。


「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。


In [5]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /root/MyResearch/AI_engineering/lecture-ai-engineering/day1


/root/MyResearch/AI_engineering/lecture-ai-engineering/day1


In [6]:
load_dotenv(find_dotenv())

True

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。


In [ ]:
%cd /content/lecture-ai-engineering/day1/01_streamlit_UI

必要なライブラリをインストールします。


In [ ]:
%%capture
!pip install -r requirements.txt

ngrok のトークンを使用して、認証を行います。


In [ ]:
!ngrok authtoken $$NGROK_TOKEN

アプリを起動します。


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開 URL の後に記載されている URL にブラウザでアクセスすると、streamlit の UI が表示されます。

app.py のコメントアウトされている箇所を編集することで、UI がどの様に変化するか確認してみましょう。

streamlit の公式ページには、ギャラリーページがあります。

streamlit を使うと python という一つの言語であっても、様々な UI を実現できることがわかると思います。

https://streamlit.io/gallery


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [ ]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。


In [14]:
%cd ./02_streamlit_app

/root/MyResearch/AI_engineering/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。


In [22]:
!pip install -r requirements.txt

ngrok と huggigface のトークンを使用して、認証を行います。


In [23]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `AI engineering class2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AI engineering class2`


stramlit で Huggingface のトークン情報を扱うために、streamlit 用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。


In [24]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

In [25]:
!export STREAMLIT_WATCH_FILE_SYSTEM=false

アプリを起動します。

02_streamlit_app では、Huggingface からモデルをダウンロードするため、初回起動には 2 分程度時間がかかります。

この待ち時間を利用して、app.py のコードを確認してみましょう。


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")

!streamlit run app.py

公開URL: https://6928-133-5-16-51.ngrok-free.app

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8503
  Network URL: http://172.19.189.241:8503

NLTK loaded successfully.
NLTK Punkt data checked/downloaded.
Database 'chat_feedback.db' initialized successfully.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  5.40it/s]
2025-04-20 19:38:45.380 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/root/miniconda3/envs/my-env/lib/python3.9/site-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/envs/my-env/lib/python3.9/site-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
  File "/root/miniconda3/envs/my-e

アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、Streamlit による UI 部分だけではなく、SQLite を使用したチャット履歴の保存や LLM のモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理の UI を提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションの UI ロジックを管理します。
- **`llm.py`**: LLM モデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLite データベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEU スコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要な Python パッケージ。


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [16]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。


In [ ]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

必要なライブラリをインストールします。


In [ ]:
%%capture
!pip install -r requirements.txt

ngrok と huggigface のトークンを使用して、認証を行います。


In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。


In [ ]:
!python app.py

FastAPI が起動すると、API とクライアントが通信するための URL（エンドポイント）が作られます。

URL が作られるのと合わせて、Swagger UI という Web インターフェースが作られます。

Swagger UI にアクセスすることで、API の仕様を確認できたり、API をテストすることができます。

Swagger UI を利用することで、API を通して LLM を動かしてみましょう。


後片付けとして、使う必要のない ngrok のトンネルを削除します。


In [ ]:
from pyngrok import ngrok
ngrok.kill()